# Суперотчет
### Библиотеки

In [1]:
from docxtpl import DocxTemplate, InlineImage
from docx.enum.dml import MSO_THEME_COLOR_INDEX
from docx.shared import Cm
import numpy as np
import pandas as pd 
from math import pi
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import requests
import re
from string import ascii_letters
import ast
import docx2txt
import json
import docx
import os
import time
import datetime
import PySimpleGUI as sg
from string import punctuation
from pymystem3 import Mystem
#from stop_words import get_stop_words
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
from matplotlib.ticker import FixedLocator
import matplotlib.pyplot as plt
from tqdm import tqdm
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from flask.json import jsonify
import matplotlib as mpltl

ModuleNotFoundError: No module named 'pkg_resources'

### Диск

In [ ]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=922428539074-3lcp6a62nd735nvdh38lj548n223snd0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


### ID папки с таблицами и видеовопросами

In [ ]:
tables_root_id = drive.ListFile({'q': "title = 'Таблицы -> Анкеты'"}).GetList()[0]['id']
videoquest_folder_names = ['Вопрос от Катерины Ходуновой', 'Вопрос от Олега Лысака']
videoquest_root_ids = [drive.ListFile({'q': "title = '{}'".format(name)}).GetList()[0]['id'] for name in videoquest_folder_names]

### Все таблицы с данными

In [ ]:
tables = []
file_list = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % tables_root_id}).GetList()
for file in file_list:
    if file['title'] == 'Шаблон отчета v.5.0a.docx':
    #if file['title'] == 'Шаблон отчета v.5.0.docx':
        print(file['title'])
        file.GetContentFile(file['title'])
        template = file['title']
    if file['mimeType'] == 'application/vnd.google-apps.spreadsheet':
        file.GetContentFile(file['title'] + '.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
        tables.append(file['title'] + '.xlsx')
    if file['mimeType'] == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        #if file['title'] == 'Тестирование. 21 вопрос.xlsx':
        #    file.GetContentFile(file['title'])
        #    tables.append(file['title'])
        if file['title'] == 'Теория технологического предпринимательства - тест.xlsx':
            file.GetContentFile(file['title'])
            tables.append(file['title'])
    if file['mimeType'] == 'text/csv':
        file.GetContentFile(file['title'])
        tables.append(file['title'])
for file_id in videoquest_root_ids:
    file = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % file_id}).GetList()[0]
    file.GetContentFile(file['title'])
    tables.append(file['title'])
tables

Шаблон отчета v.5.0a.docx


['Nice2MeetYou.xlsx',
 'QuestionsFromStudents.xlsx',
 'Cases.xlsx',
 'Game.xlsx',
 'Knowledge.xlsx',
 'TestOpinion.xlsx',
 'Теория технологического предпринимательства - тест.xlsx',
 'Вопрос+от+Катерины+Ходуновой.xlsx',
 'Вопрос+от+Олега+Лысака.xlsx']

### Таблицы к DataFrame

In [ ]:
dfs = []
for table in tables:
    df = pd.read_excel(table, dtype = str, keep_default_na = False)
    if 'Потрачено времени' in df.columns:
        df = df.iloc[1:]
        df['Процент правильных ответов (%)'] = df['Процент правильных ответов (%)'].apply(float).round().apply(int).apply(str)
        df.rename(columns = {'Потрачено времени': 'time', 'Процент правильных ответов (%)': 'score_test'}, inplace = True)
    df.columns = [col.replace(' ', '').replace('/', '').replace('.', '').lower() for col in df.columns]
    dfs.append(df)

### Получение имен, городов

In [ ]:
index_table = [tab for tab in tables if tab.startswith('Nice')][0]
df0 = pd.read_excel(index_table, dtype = str, keep_default_na = False)
df0['gorodprojivaniya'].replace('/', ';', regex=True, inplace=True)
df0['ma_email'] = df0['ma_email'].str.lower()
emails = df0['ma_email'][:].unique()
emails

array(['bgdshv.a@gmail.com', 'elashkina.anna@gmail.com',
       'fedoseev.is@enrubattery.ru', ..., 'fejlucksfate@gmail.com',
       'den.kustov.03@mail.ru', 'polly.2003t@gmail.com'], dtype=object)

### Колонка почты

In [ ]:
def email_col(df):
    return max([col for col in df.columns if 'mail' in col], key=len)

## Первичный балл

### Словарь баллов

In [1]:
primary_keys_dict = {'chtoudivilomaterials': 5, 'chtorazozlilomaterials': 5,
                                'vopros1': 1, 'vopros2': 1, 'vopros3': 1,
                                'q1': 4, 'q2': 4,
                                'question': 3}
primary_cases_list = ['case1', 'case2', 'case3', 'case4']

### Токенизация

In [2]:
# Lemmatizer and stopwords
mystem = Mystem() 
#stop_words = set(get_stop_words('russian'))
stop_words = {'восьмой', 'многочисленное', 'свою', 'нх', 'наше', 'ними', 'тут', 'нею', 'мною', 'чего', 'перед', 'по', 'будто', 'вверх', 'мы', 'этой', 'мне', 'четырнадцать', 'иметь', 'будет', 'близко', 'одного', 'теперь', 'собою', 'свои', 'восемнадцатый', 'ли', 'зато', 'мож', 'сказал', 'пятнадцать', 'сих', 'важные', 'разве', 'твоя', 'сами', 'без', 'почти', 'тринадцатый', 'почему', 'об', 'семнадцатый', 'лет', 'многочисленные', 'ну', 'ещё', 'всею', 'вами', 'также', 'конечно', 'ему', 'вас', 'восемь', 'над', 'года', 'мои', 'том', 'ваши', 'или', 'меля', 'больше', 'раньше', 'самими', 'стал', 'того', 'мог', 'важная', 'же', 'та', 'ничего', 'еще', 'можхо', 'сказала', 'да', 'десятый', 'затем', 'ж', 'процентов', 'другие', 'вниз', 'лучше', 'ее', 'твой', 'во', 'ни', 'ней', 'говорил', 'довольно', 'с', 'надо', 'сколько', 'всё', 'двадцать', 'самим', 'уже', 'занято', 'низко', 'другой', 'двенадцатый', 'всегда', 'которой', 'своей', 'вдали', 'у', 'до', 'его', 'наши', 'мочь', 'отовсюду', 'нужно', 'долго', 'две', 'меня', 'одиннадцать', 'тебя', 'но', 'сегодня', 'он', 'которые', 'кроме', 'этими', 'вам', 'какой', 'бывает', 'будете', 'за', 'девять', 'недалеко', 'более', 'самому', 'её', 'каждая', 'рано', 'всю', 'однажды', 'сейчас', 'девятнадцатый', 'важный', 'многочисленный', 'себя', 'совсем', 'сказать', 'эту', 'всему', 'про', 'занята', 'вся', 'имя', 'бы', 'нам', 'одной', 'миллионов', 'тех', 'туда', 'дел', 'три', 'уж', 'внизу', 'оба', 'моё', 'их', 'чему', 'кругом', 'раз', 'кому', 'мимо', 'сначала', 'эта', 'прекрасно', 'кого', 'которая', 'нельзя', 'где', 'была', 'рядом', 'те', 'всех', 'десять', 'везде', 'каждые', 'особенно', 'никуда', 'ваш', 'иногда', 'мало', 'неё', 'что', 'жизнь', 'между', 'однако', 'пожалуйста', 'которых', 'многочисленная', 'всем', 'могут', 'хотя', 'этим', 'сама', 'тринадцать', 'чтобы', 'ком', 'несколько', 'уметь', 'всюду', 'должно', 'самой', 'теми', 'чуть', 'каждый', 'наша', 'этот', 'седьмой', 'я', 'кто', 'эти', 'давно', 'нибудь', 'которого', 'такое', 'все', 'значит', 'тем', 'тому', 'а', 'четырнадцатый', 'наш', 'сам', 'меньше', 'этом', 'моя', 'снова', 'недавно', 'пять', 'этого', 'то', 'позже', 'потому', 'для', 'человек', 'из', 'наверху', 'потом', 'говорит', 'других', 'наконец', 'вы', 'начала', 'второй', 'после', 'бывь', 'будут', 'часто', 'она', 'самих', 'семь', 'году', 'этому', 'саму', 'им', 'какая', 'своих', 'ими', 'при', 'нему', 'никогда', 'двадцатый', 'тот', 'четыре', 'девятый', 'восемнадцать', 'люди', 'них', 'около', 'мой', 'двенадцать', 'четвертый', 'будешь', 'на', 'чтоб', 'шестнадцать', 'вдруг', 'этих', 'тобой', 'ты', 'хотеть', 'как', 'мной', 'буду', 'было', 'шесть', 'мира', 'и', 'пока', 'даже', 'здесь', 'непрерывно', 'были', 'ниже', 'всего', 'само', 'ту', 'своего', 'такие', 'нет', 'двух', 'в', 'свое', 'будь', 'под', 'вокруг', 'пор', 'пятнадцатый', 'твоё', 'был', 'г', 'времени', 'куда', 'шестой', 'если', 'будем', 'такая', 'есть', 'может', 'пора', 'оно', 'далеко', 'немного', 'один', 'отсюда', 'такой', 'ваше', 'тебе', 'через', 'ваша', 'девятнадцать', 'чаще', 'хорошо', 'там', 'тогда', 'о', 'нередко', 'очень', 'лишь', 'опять', 'тысяч', 'ей', 'тобою', 'назад', 'тою', 'шестнадцатый', 'весь', 'спасибо', 'суть', 'него', 'пятый', 'одиннадцатый', 'время', 'дальше', 'друго', 'чем', 'не', 'ведь', 'вот', 'хочешь', 'так', 'даром', 'самом', 'можно', 'себе', 'много', 'посреди', 'занят', 'это', 'всеми', 'день', 'действительно', 'просто', 'важное', 'менее', 'третий', 'сеаой', 'нас', 'алло', 'который', 'самого', 'обычно', 'вон', 'год', 'нее', 'кем', 'хоть', 'нем', 'другое', 'первый', 'именно', 'м', 'они', 'т', 'быть', 'мор', 'два', 'когда', 'слишком', 'к', 'ею', 'кажется', 'от', 'семнадцать', 'со', 'е', 'против', 'зачем', 'впрочем', 'каждое', 'тоже', 'нами', 'только', 'заняты', 'другая', 'собой', 'наиболее'}

# Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(re.sub(r'\d+', '', text.lower()))
    tokens = [token for token in tokens if token not in stop_words
              and token != " "
              and token.strip() not in punctuation]
    
    return list(dict.fromkeys(tokens))

def case_counter(text, case):
    tokens = preprocess_text(text)
    
    return sum(word in dict_cases[case] for word in tokens)

NameError: ignored

### Словари кейсов

In [ ]:
dict_cases = {
    'case1': "представлять предприниматель компания заниматься разработка высокоточный станок производство пробный партия необходимо купить специальный клей исследовать возможный предложение интернет понимать поставка организовывать германия необходимый количество стоять евро самостоятельно приходиться сделать следующий заплатить закупка доставка налог пошлина потратить рабочий оформление документ ждать счастие инженер отец жить близкий собираться навещать сын заказывать домой привозить багаж неделя действовать официальный воспользоваться удачный стечение обстоятельство написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case2': "представлять коммерческий директор компания разработка отопительный котел новый поколение месяц выпускать прототип показывать заказчик огромный строительный фирма оставаться довольный сделать следующий предложение готовый купить экземпляр пробный запускать достраивать здание проба продлиться оказываться годный сразу партия знать рынок конкурент каков особенность продукция данный момент участвовать испытание соглашаться отдавать начинать оформлять тз производство штука контрактный главный разработчик сообщать находить решение повышать производительность переделывать опытный образец конструкторский бюро проверять тестовый стенд число поменять короб помещаться отправлять устаревать конструкция инженер говорить срочно предлагать вернуть тестирование утверждать  % ,  вариант причина выбирать стоять вопрос обращаться прерывать середина путь технический характеристика продукт рискнуть потратить деньги подготовка документация улучшать точно скоро хороший примета складываться ситуация написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case3': "представлять молодой предприниматель открывать небольшой стартап партнерство одногруппник разрабатывать крутой приложение смартфон уверять взлетать пользоваться огромный спрос очередной этап разработка понимать свой деньги хватать нужный инвестиция нанимать дополнительный разработчик дизайнер тестировщик покупать лицензия облако место надежный сервер посчитать близкий находить млн рубль неделя поиск приходить кредитный предложение банк предлагать вполне сносный условие взять льготный годовой получать перевод завтра отвечать венчурный фонд mobileventures согласный проинвестировать транш входить доля ждать результат месяц возможный следующий раунд переговоры партнер встречаться двоюродный дядя рассказывать идея понравиться склон скопить приличный сумма заниматься поставка продукт питание из-за рубеж готовый вкладывать единственный поставлять держать курс дело принятие решение интересно следить строиться новый технология поступать источник вообще написать цепочка рассуждение подробно аргументировать ответ".split(' '),
    'case4': "представлять начальник департамент инновация крупный корпорация подобный ibm усиливать инновационность деятельность собственно продукт достаточно большой бюджет поставлять амбициозный задача ряд проектный сессия вариант деньги выделять хватить начинать инвестировать самостоятельно фонд молодой стартапа нужный направление ожидать выходить улучшать повышать эффективность работа самый инвестиция предполагать получение доля компания должный возвращать последствие либо полностью выкупать процент успешность оценивать предлагать сотрудник сумма экономия приносить идея обязывать гарантировать рассмотрение поступать обоснованный ответ случай отказ принимать  –  таков корпоративный этика решение совещание совет директор".split(' ')
    }

### Словари новеллы

In [ ]:
q_scored = ['q23', 'q50', 'q34', 'q41', 'q48', 'q62', 'q70']
q_translated = ['q10', 'q18', 'q33', 'q40', 'q45', 'q48', 'q59', 'q60', 'q61', 'q70']

### ID папки с новеллой

In [ ]:
novella_root_id = drive.ListFile({'q': "title = 'НовеллаБД'"}).GetList()[0]['id']

### Загружаем данные о новелле

In [ ]:
file_list = drive.ListFile({'q': " '%s' in parents and trashed = FALSE" % novella_root_id}).GetList()
for file in file_list:
    if file['mimeType'] == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        file.GetContentFile(file['title'])
        rules = pd.read_excel(file['title'])
    #if file['mimeType'] == 'text/csv':
    #    file.GetContentFile(file['title'])
    #    data = pd.read_csv(file['title'], sep = ";", encoding="windows-1251", quotechar = '"')
url = 'https://10education.ru'
r = requests.get(url)
first_list = r.text.split('<br />')
index_pass = first_list[0].index('/html')
first_list[0] = first_list[0][index_pass+10:]
second_list = [string.split(' - ') for string in first_list]
df = pd.DataFrame(second_list)
columns_names = {0: 'username', 1: 'email', 2: 'password', 3: 'history_new'}
data = df.rename(columns=columns_names)
data['ma_email'] = data['email']
data = data[:-1]

### Добавляем данные о новелле

In [ ]:
def novella_insert(email, data, rules):
    if data[data['email'] == email]['history_new'].to_list() != []:
        h_str = data[data['email'] == email]['history_new'].to_list()[0]
        h_json = json.loads(h_str)
        h_list = [rec for rec in h_json.values() if rec != {}]
        if h_list == []:
            return('-', '-', '-', '-', '-')
        h_record = max(h_list, key=max)
        novella_score = len(set(h_record.keys()).intersection(q_scored)) * 5
        if 'q19' in h_record.keys() and h_record['q19'] == 'a1':
            novella_score += 5
        if 'q37' in h_record.keys() and h_record['q37'] == 'a2':
            novella_score += 5
        if "Использовать старые данные" in h_record.values():
            novella_score += 5
        novella_translated = []
        novella_last = 'пропуск'
        novella_pizza = np.array([0, 0, 0, 0, 0, 0, 0, 0])

        for key, value in h_record.items():
            if key in rules['Вопрос'].to_list() and value in rules.columns.to_list():
                rules_row = (rules[rules['Вопрос'] == key])
                value_new = rules_row[value+'_axial_value'].to_list()[0]
                try:
                    novella_pizza += np.array(ast.literal_eval(value_new))
                except:
                    pass
                if key in q_translated:
                    value_translated = rules_row[value+'_i'].to_list()[0]
                    novella_translated.append(value_translated)
        novella_translated = " ".join(novella_translated)

        try:
            for pair in list(h_record.items())[::-1]:
                if pair[0].startswith('q'):
                    key_last, value_last = pair
                    rules_row = (rules[rules['Вопрос'] == key_last])
                    key_new = rules_row['Перевод'].to_list()[0]
                    value_new = rules_row[value_last+'_i'].to_list()[0]
                    novella_last = value_new
                    break
        except:
            pass

        if 'q21' in h_record.keys():
            novella_branch = 'Корпоративная'
        elif 'q49' in h_record.keys():
            novella_branch = 'Торговая'
        else:
            novella_branch = '-'

        return(novella_score, novella_translated, novella_last, novella_branch, novella_pizza)
    else:
        return('-', '-', '-', '-', '-')

### Пицца - граф

In [ ]:
def radar_factory(num_vars, frame='circle'):
    """Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle' | 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarAxes(PolarAxes):

        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return Circle((0.5, 0.5), 0.5)
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            """ Draw. If frame is polygon, make gridlines polygon-shaped """
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().rotate(pi/8).scale(0.5).translate(.5, .5)
                                    + self.transAxes)

                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

### Вставка пиццы

In [ ]:
col1 = 'Дисконтирование\nустаревших\nрешений'
col2 = 'Давление на\nнезависимого\nпартнера'
col3 = 'Конструирование\n компаний'
col4 = 'Препарирование\nтехнологий'
col5 = 'Скрининг\nшансов'
col6 = 'Борьба за\nпривилегии'
col7 = 'Фиктивная\nдеятельность'
col8 = 'Отношения\nвместо дела'
zero = 0

def web(doc, numbers, k):
    dict1 = {col : zero for col in [col1, col2, col3, col4, col5, col6, col7, col8]}
    N = 8
    theta = radar_factory(8, frame='polygon')

    angles = np.linspace(0, 2 * pi, N, endpoint=False)
    angles_mids = angles + (angles[1] / 2)
    #spoke_labels = data.pop(0)
    #title, case_data = data[0]

    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(left=0)
    ax.xaxis.set_minor_locator(FixedLocator(angles))
    fig.subplots_adjust(top=0.85, bottom=0.05)
    plt.ylim([0, 10])

    ax.set_theta_offset(pi/8-pi/4)
    ax.set_theta_direction(1)
    ax.set_xticks(angles_mids)
    ax.set_xticklabels(dict1.keys())
    ax.grid(True, axis='x', which='minor')
    ax.grid(False, axis='x', which='major')
    ax.grid(True, axis='y', which='major')
    ax.set_rgrids([2,4,6,8,10], [])
    #ax.set_title(file_name,  position=(0.5, 1.1), ha='center')
    color1='springgreen'
    color2='seagreen'
    color3='tomato'
    color4='red'
    for i in range(6):
        ax.fill_between(angles[i:i+2], 0, numbers[i] , facecolor=color1, edgecolor=color2, linewidth=2)
    ax.fill_between(angles[5:7], 0, numbers[5], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between(angles[6:8], 0, numbers[6], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between([angles[7],(angles[7]+angles[1])], 0, numbers[7], facecolor=color3, edgecolor=color4, linewidth=2)

    for index, label in enumerate(ax.get_xticklabels()):
        label.set_weight("bold")
        label.set_fontsize(14)
        if index in [0, 1, 7]:
            label.set_horizontalalignment("left")
        elif index in [3, 4, 5]:
            label.set_horizontalalignment("right")

    plt.savefig('web{}.png'.format(str(k)), transparent=True)
    return InlineImage(doc, 'web{}.png'.format(str(k)), Cm(10))

### Подсчет баллов

In [ ]:
def primary_point(context):
    if 'score' in context:
        pts = 0.2 * max([int(num) for num in context['score'].split()])
    else:
        pts = 0
    for key, value in primary_keys_dict.items():
        if key in context.keys() and len(context[key]) > 10:
            pts += value
    pts_c = 0
    for key in primary_cases_list:
        if key in context.keys():
            if len(context[key]) >= 50:
                pts_c += 1
            if len(context[key]) >= 300:
                pts_c += 2
            counter = case_counter(context[key], key)
            if counter >= 4:
                pts_c += 2
            if counter >= 6:
                pts_c += 1
            if counter >= 8:
                pts_c+= 1
    pts += pts_c
    if context['novel_score'] != '-':
        pts += context['novel_score']
        if pts > 100:
            pts = 100
            
    return round(pts), pts_c

### Подсчет баллов пицца

In [ ]:
cases_pizza_low = {'case1': np.array([0.5, 0, 0.5, 0, 1, 0, 0 ,0]), 'case2': np.array([0, 0.5, 0.5, 1.5, 0, 0, 0 ,0]), 'case3': np.array([0, 1, 0, 0, 0.5, 0, 0 ,0])}
cases_pizza_high = {'case1': np.array([1, 0, 1, 0, 2, 0, 0 ,0]), 'case2': np.array([0, 1, 1, 3, 0, 0, 0 ,0]), 'case3': np.array([0, 2, 0, 0, 1, 0, 0 ,0])}

def score_cases_pizza(context):
    score_cases = np.array([0, 0, 0, 0, 0, 0, 0 ,0])
    if 'score' in context:
        if int(context['score']) >= 90:
            score_pizza = np.array([0, 0, 2, 1, 0, 0, 0, 0])
        elif int(context['score']) >= 60:
            score_pizza = np.array([0, 0, 1, 0, 0, 0, 0, 0])
        else:
            score_pizza =  np.array([0, 0, 0, 0, 0, 0, 0 ,0])
        score_cases += score_pizza
    for key in cases_pizza_low.keys():
        pts =  0
        if key in context.keys():
            if len(context[key]) >= 50:
                pts += 1
            if len(context[key]) >= 300:
                pts += 2
            counter = case_counter(context[key], key)
            if counter >= 4:
                pts += 2
            if counter >= 6:
                pts += 1
            if counter >= 8:
                pts += 1
        if pts >= 6:
            score_case = cases_pizza_high[key]
        if pts >= 3:
            score_case = cases_pizza_low[key]
        else:
            score_case = np.array([0, 0, 0, 0, 0, 0, 0 ,0])
        score_cases = score_cases + score_case
       # print(key, score_case, score_cases)
            
    return score_cases

## Новые анкеты

### Проверка наличия концовки

In [ ]:
def novella_check_end(email):
    if data[data['email'] == email]['history_new'].to_list() != []:
        h_str = data[data['email'] == email]['history_new'].to_list()[0]
        h_json = json.loads(h_str)
        h_list = [rec for rec in h_json.values() if rec != {}]
        if h_list == []:
            return("-")
        h_record = max(h_list, key=max)
        if "ending" in h_record.keys() and len(h_list) > 1:
            return("++")
        elif "ending" in h_record.keys():
            return("+")
        else:
            return("-")
    else:
        return("/")

### Анализ новеллы

In [ ]:
def novella_check(email):
    if data[data['email'] == email]['history_new'].to_list() != []:
        h_str = data[data['email'] == email]['history_new'].to_list()[0]
        h_json = json.loads(h_str)
        h_list = [rec for rec in h_json.values() if rec != {}]
        if h_list == []:
            return("/")
        h_record = max(h_list, key=max)
    else:
        return("/")
        
    try:
        for pair in list(h_record.items())[::-1]:
            if pair[0].startswith('q'):
                break
    except:
        return("/")
    if pair == ('q21', 'a1'):
        return("Участник не мотивирован к переходу в предпринимательскую позицию.")
    elif pair[0] in ['q29', 'q27', 'q54', 'q56'] and pair[1] == 'a1':
        return("Участник испытывает трудности с удержанием предпринимательской позиции.")
    elif pair == ('q41', 'a1') or pair == ('q62', 'a1'):
        return("Участник склонен к нереалистичным ожиданиям от предпринимательской деятельности.")
    elif any([q in h_record.keys() for q in ['q48', 'q70']]):
        return("Участник демонстрирует высокий потенциал предпринимательских компетенций.")
    else:
        return("/")

### Заполнение анкет

In [ ]:
def context_check(context):
    context_checked = context
    keys = ['name', 'email', 'phone', 'socialmedia', 'univer', 'futureplans', 'gorodprojivaniya']
    #for key in keys:
    #    context_checked[key] = context[key]
    context_checked['comp_1'] = "/"
    context_checked['comp_2'] = "/"
    context_checked['comp_3'] = "+" if context['score'] >=60 else "-"
    context_checked['comp_4'] = "+" if context['novel_score'] != '-' and context['novel_score'] >=10 else "-"
    context_checked['comp_5'] = "++" if context['cases_score'] >=24 else "+" if context['cases_score'] >=16 else "-"
    context_checked['comp_6'] = "+" if all([context[q] != 'пропуск' for q in ['q1', 'q2']]) else "-"
    context_checked['comp_t'] = "+" if sum([context_checked['comp_'+str(i)].count('+') for i in range(1,7)]) >=3 else "-"
    
    moti_1_1, moti_1_2 = [context[key] != 'пропуск' for key in keys], len(context['futureplans'])
    if all(moti_1_1) and moti_1_2 > 200:
        context_checked['moti_1'] = "++"
    elif moti_1_1.count(False) == 1:
        context_checked['moti_1'] = "+"
    else:
        context_checked['moti_1'] = "-"
        
    moti_2_1 = all([len(context[key])>200 for key in ['chtoudivilomaterials', 'chtorazozlilomaterials']])
    moti_2_2 = [context[key] != 'пропуск' for key in ['chtoudivilomaterials', 'chtorazozlilomaterials']]
    if moti_2_1:
        context_checked['moti_2'] = "++"
    elif moti_2_2:
        context_checked['moti_2'] = "+"
    else:
        context_checked['moti_2'] = "-"
        
    moti_3_1 = max([sum([a*b for a,b in zip([3600,60,1], map(int, d.split(':')))]) for d in context['time'].split()]) \
        if context['time'] != 'пропуск' else 0
    if moti_3_1 > 1200:
        context_checked['moti_3'] = "++"
    elif moti_3_1 > 600:
        context_checked['moti_3'] = "+"
    else:
        context_checked['moti_3'] = "-"
        
    context_checked['moti_4'] = novella_check_end(context['email'])
    context_checked['moti_5'] = "++" if context['cases_score'] >=24 else "+" if context['cases_score'] >=16 else "-"
    context_checked['moti_6'] = "+" if any([context[q] != 'пропуск' for q in ['q1', 'q2']]) else "-"
    context_checked['moti_t'] = "+" if sum([context_checked['moti_'+str(i)].count('+') for i in range(1,7)]) >=4 else "-"    
    
    comp_1_1, comp_1_2 = all([context[key] != 'пропуск' for key in keys]), len(context['futureplans'])
    if comp_1_1 and comp_1_2 > 200:
        context_checked['comm_1'] = "Демонстрирует мотивацию к участию и отвественность в работе с формами сбора данных."
    elif comp_1_1 and comp_1_2 > 50:
        context_checked['comm_1'] = "Демонстрирует мотивацию к участию."
    else:
        context_checked['comm_1'] = "/"
    
    comp_2_1 = max([int(s[0]) for s in context['vybormateriala'].split() if s[0].isdigit()]+[0]) \
        if context['vybormateriala'] != 'пропуск' else 0
    comp_2_2 = all([len(context[key])>200 for key in ['chtoudivilomaterials', 'chtorazozlilomaterials']])
    comp_2_3 = [int(s) for s in context['score_test'].split() if s.isdigit()]
    if comp_2_1 > 3 and comp_2_2 and len(comp_2_3)==1 and comp_2_3[0] > 80:
        context_checked['comm_2'] = "Демонстрирует высокие мотивацию и способность к освоению новой информации"
    elif comp_2_1 > 3 and comp_2_2:
        context_checked['comm_2'] = "Демонстрирует мотивацию к участию."
    else:
        context_checked['comm_2'] = "/"
        
    if len(comp_2_3)>1:
        comp_3_1, comp_3_2 = comp_2_3, context['time'].split()
        comp_3_3 = [comp_3_2[i] for i in [comp_3_1.index(n) for n in comp_3_1 if n>comp_3_1[-1]]]
        comp_3_4 = any([sum([a*b for a,b in zip([3600,60,1], map(int, d.split(':')))]) >300 for d in comp_3_3])
        context_checked['comm_3'] = "Участник имеет склонность к фиктивной деятельности"
    else:
        context_checked['comm_3'] = "/"
    
    context_checked['comm_4'] = novella_check(context['email'])
    
    if context_checked['comp_5'] == "+" and context_checked['moti_5'] == "+":
        context_checked['comm_5'] = "Участник демонстрирует хорошее понимание предпринимательской логики."
    elif context_checked['moti_5'] == "+":
        context_checked['comm_5'] = "Участнику не хватает прикладного препрдпринимательского опыта."
    else:
        context_checked['comm_5'] = "/"
        
    context_checked['comm_6'] = "Участник демонстрирует высокий уровень персональной ответственности" if all([context[q] != 'пропуск' for q in ['q1', 'q2']]) else "/"
    
    if all([x=="+" for x in [context_checked['comp_t'], context_checked['moti_t']]]):
        echelon = 1
    elif any([x=="+" for x in [context_checked['comp_t'], context_checked['moti_t']]]):
        echelon = 2
    else:
        echelon = 3
    return(context_checked, echelon)

## Окно

In [ ]:
def window_string(key, text_size, input_size, tooltip=None):
    return ([sg.Text(key, size=(text_size,1)), sg.InputText(key=key, size=(input_size,1), tooltip=tooltip)])

def make_window():    
    tab1_layout = [
        [sg.HorizontalSeparator()],
        window_string('Номер договора', 26, 30),
        window_string('Число подписания договора', 26, 30, 'числом'),
        window_string('Месяц подписания договора', 26, 30, 'словом'),
        window_string('День начала', 26, 30, 'числом'),
        window_string('Месяц начала', 26, 30, 'словом'),
        window_string('День окончания', 26, 30, 'числом'),
        window_string('Месяц окончания', 26, 30, 'словом'),
        [sg.HorizontalSeparator()],
        window_string('Должность подписывающего', 26, 30, 'со стороны заказчика'),
        window_string('Краткое название заказчика', 26, 30),
        window_string('ФИО подписывающего', 26, 30, 'со стороны заказчика'),
        window_string('Договор заключён с ', 26, 30),
        window_string('Полная стоимость услуг', 26, 30, 'цифрами'),
        window_string('Полная стоимость услуг словами', 26, 30),
        window_string('День отчета', 26, 30, 'числом'),
        window_string('Месяц отчета', 26, 30, 'словом'),
        [sg.HorizontalSeparator()]
        ]
    
    tab2_layout = [
        [sg.HorizontalSeparator()],
        [sg.Text('Ключ', size=(26, 1)), sg.Combo(sorted(dict_key.keys()), readonly=True, key='-KEY-', size=(30, 10))],
        [sg.Checkbox('Факультет', default=False, key='add_id_1')],
        [sg.Checkbox('Курс', default=False, key='add_id_2')],
        window_string('Название папки на диске', 26, 30),
        [sg.Text('Асессор заключение', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='conclusion'), sg.InputText(size=(30, 1), key='conclusion')],
        [sg.ProgressBar(100, orientation='h', size=(40,20), key='progress_bar')],
        [sg.HorizontalSeparator()],
        [sg.Output(size=(50,6))],
        [sg.Submit('Запустить'), sg.Cancel('Выйти')],
        [sg.HorizontalSeparator()]
        ]
    
    tab_group_layout = [[sg.Tab('Строки', tab1_layout, key='-TAB1-'),
                                      sg.Tab('Ключ', tab2_layout, key='-TAB2-')]]

    layout = [[sg.TabGroup(tab_group_layout,
                               enable_events=True,
                               key='-TABGROUP-')]]
    
    return sg.Window('Суперотчет v0.7', layout)

### Запись на диск

In [ ]:
def drive_folder_create(parent_id, folder_name):
    file = drive.CreateFile({'title': folder_name,
        "parents": [{"id": parent_id}],
        "mimeType": "application/vnd.google-apps.folder"})
    file.Upload()
    permission = file.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    return(file)

def drive_file_create(file_name, parent_id, file_path):
    file = drive.CreateFile({
        'title': file_name, 
        "parents": [{"id": parent_id}]
        })
    file.SetContentFile(file_path)
    file.Upload()
    permission = file.InsertPermission({
                    'type': 'anyone',
                    'value': 'anyone',
                    'role': 'reader'})
    return(file['alternateLink'])

### + Электронное письмо

In [ ]:
def send():
    # create message object instance
    msg = MIMEMultipart()
    
    message = 'Done!'
    # setup the parameters of the message
    password = "nc2016ts"
    msg['From'] = "buildcompanyarchive@gmail.com"
    msg['To'] = "10markmarkmark10@gmail.com"
    msg['Subject'] = "Test Mail"

    # add in the message body
    msg.attach(MIMEText(message, 'plain'))

    #create server
    server = smtplib.SMTP('smtp.gmail.com: 587')

    server.starttls()

    # Login Credentials for sending the mail
    server.login(msg['From'], password)

    # send the message via the server.
    server.sendmail(msg['From'], msg['To'], msg.as_string())

    server.quit()
    
    return message

## Вывод

In [ ]:
def context_check(context):
    context_checked = context
    keys = ['name', 'email', 'phone', 'socialmedia', 'univer', 'futureplans', 'gorodprojivaniya']
    #for key in keys:
    #    context_checked[key] = context[key]
    context_checked['comp_1'] = "/"
    context_checked['comp_2'] = "/"
    context_checked['comp_3'] = "++" if context['score'] >=60 else "+" if context['score'] >= 40 else "-"
    context_checked['comp_4'] = "++" if context['novel_score'] != '-' and context['novel_score'] >=10 else "+" \
                                    if context['novel_score'] != '-' and context['novel_score'] >=5 else "-"
    context_checked['comp_5'] = "++" if context['cases_score'] >=24 else "+" if context['cases_score'] >=12 else "-"
    context_checked['comp_6'] = "+" if all([context[q] != 'пропуск' for q in ['q1', 'q2']]) else "-"
    context_checked['comp_t'] = "+" if sum([context_checked['comp_'+str(i)].count('+') for i in range(1,7)]) >=3 else "-"
    
    moti_1_1, moti_1_2 = [context[key] != 'пропуск' for key in keys], len(context['futureplans'])
    if all(moti_1_1) and moti_1_2 > 100:
        context_checked['moti_1'] = "++"
    elif moti_1_1.count(False) == 1:
        context_checked['moti_1'] = "+"
    else:
        context_checked['moti_1'] = "-"
        
    moti_2_1 = all([len(context[key])>100 for key in ['chtoudivilomaterials', 'chtorazozlilomaterials']])
    moti_2_2 = [context[key] != 'пропуск' for key in ['chtoudivilomaterials', 'chtorazozlilomaterials']]
    if moti_2_1:
        context_checked['moti_2'] = "++"
    elif moti_2_2:
        context_checked['moti_2'] = "+"
    else:
        context_checked['moti_2'] = "-"
        
    moti_3_1 = max([sum([a*b for a,b in zip([3600,60,1], map(int, d.split(':')))]) for d in context['time'].split()]) \
        if context['time'] != 'пропуск' else 0
    if moti_3_1 > 1200:
        context_checked['moti_3'] = "++"
    elif moti_3_1 > 600:
        context_checked['moti_3'] = "+"
    else:
        context_checked['moti_3'] = "-"
        
    context_checked['moti_4'] = novella_check_end(context['email'])
    context_checked['moti_5'] = "++" if context['cases_score'] >=24 else "+" if context['cases_score'] >=12 else "-"
    context_checked['moti_6'] = "+" if any([context[q] != 'пропуск' for q in ['q1', 'q2']]) else "-"
    context_checked['moti_t'] = "+" if sum([context_checked['moti_'+str(i)].count('+') for i in range(1,7)]) >=4 else "-"    

    comp_1_1, comp_1_2 = all([context[key] != 'пропуск' for key in keys]), len(context['futureplans'])
    if comp_1_1 and comp_1_2 > 200:
        context_checked['comm_1'] = "Демонстрирует мотивацию к участию и отвественность в работе с формами сбора данных."
    elif comp_1_1 and comp_1_2 > 50:
        context_checked['comm_1'] = "Демонстрирует мотивацию к участию."
    else:
        context_checked['comm_1'] = "/"
    
    comp_2_1 = max([int(s[0]) for s in context['vybormateriala'].split() if s[0].isdigit()]+[0]) \
        if context['vybormateriala'] != 'пропуск' else 0
    comp_2_2 = all([len(context[key])>200 for key in ['chtoudivilomaterials', 'chtorazozlilomaterials']])
    comp_2_3 = [int(s) for s in context['score_test'].split() if s.isdigit()]
    if comp_2_1 > 3 and comp_2_2 and len(comp_2_3)==1 and comp_2_3[0] > 80:
        context_checked['comm_2'] = "Демонстрирует высокие мотивацию и способность к освоению новой информации"
    elif comp_2_1 > 3 and comp_2_2:
        context_checked['comm_2'] = "Демонстрирует мотивацию к участию."
    else:
        context_checked['comm_2'] = "/"
        
    if len(comp_2_3)>1:
        comp_3_1, comp_3_2 = comp_2_3, context['time'].split()
        comp_3_3 = [comp_3_2[i] for i in [comp_3_1.index(n) for n in comp_3_1 if n>comp_3_1[-1]]]
        comp_3_4 = any([sum([a*b for a,b in zip([3600,60,1], map(int, d.split(':')))]) >300 for d in comp_3_3])
        context_checked['comm_3'] = "Участник имеет склонность к фиктивной деятельности"
    else:
        context_checked['comm_3'] = "/"
    
    context_checked['comm_4'] = novella_check(context['email'])
    
    if context_checked['comp_5'] == "+" and context_checked['moti_5'] == "+":
        context_checked['comm_5'] = "Участник демонстрирует хорошее понимание предпринимательской логики."
    elif context_checked['moti_5'] == "+":
        context_checked['comm_5'] = "Участнику не хватает прикладного препрдпринимательского опыта."
    else:
        context_checked['comm_5'] = "/"
        
    context_checked['comm_6'] = "Участник демонстрирует высокий уровень персональной ответственности" if all([context[q] != 'пропуск' for q in ['q1', 'q2']]) else "/"
    
    if all([x=="+" for x in [context_checked['comp_t'], context_checked['moti_t']]]):
        echelon = 1
    elif any([x=="+" for x in [context_checked['comp_t'], context_checked['moti_t']]]):
        echelon = 2
    else:
        echelon = 3
    return(context_checked, echelon)

### Общий поиск и запись анкет

In [ ]:
def main(window, values):
    middle_time = time.time()
    folder = drive_folder_create(folder_root_id, values['Название папки на диске'] if values['Название папки на диске'] != '' else 'Новая папка')
    folder_id, folder_link = folder['id'], folder['alternateLink']
    values['folder_link'] = folder_link
    folder_id_1 = drive_folder_create(folder_id, '1й эшелон')['id']
    folder_id_2 = drive_folder_create(folder_id, '2й эшелон')['id']
    
    emails = df0[df0['referer'] == dict_key[values['-KEY-']]]['ma_email'].unique()
    emails = [email for email in emails if 'test' not in email]
    dict_col = []
    keys = re.findall('{{(.+)}}', docx2txt.process(template))
    keys.append('q1videoaudiourls')
    result = []
    items = []
    folder_dict = {1: folder_id_1, 2: folder_id_2}
    tables = {1: [], 2: [], 3: [], 'final': []}
    echelons = {1: [], 2: []}
    k = 0
    
    dict_col_labels = {'k': '№', 'name_2': 'Фамилия', 'name': 'Имя', 'name_3': 'Отчество', 'email': 'Почта', 'phone': 'Телефон'}
    col_labels = dict_col_labels.keys()    
    if values['add_id_1'] and values['add_id_2']:
        values['key'] = 2
        dict_col_labels.update({'univer_2': 'Факультет', 'univer': 'Курс'})
        keys += ['univer_2', 'univer']
    elif values['add_id_1']:
        values['key'] = 1
        values['add_id'], values['ban_id'] = 'Факультет', 'univer'
        dict_col_labels.update({'univer_2': 'Факультет'})
        keys += ['univer_2']
    elif values['add_id_2']:
        values['key'] = 1
        values['add_id'], values['ban_id'] = 'Курс', 'univer_2'
        dict_col_labels.update({'univer': 'Курс'})
        keys += ['univer']
    else:
        values['key'] = 0
        
    try:
        doc_conclusion = docx.Document(values['conclusion'])
        fullText = []
        for para in doc_conclusion.paragraphs:
            fullText.append(para.text)
        if 'Пример общего заключения асессора:' in fullText:
            fullText = fullText[:fullText.index('Пример общего заключения асессора:')]
        values['conclusion'] = fullText[1:]
    except:
        values['conclusion'] = ''
            
    #doc = DocxTemplate("Ввод/Практикум_шаблон {}.docx".format(values['key']))
    doc = DocxTemplate("Ввод/Практикум_шаблон_экстренный.docx ")
    for key in list(values.keys()):
        if isinstance(key, str) and isinstance(values[key], str):
            key_new = key.replace(" ", "_")
            values[key] = values[key].replace("Нет", "")
            values[key_new] = values.pop(key)
    for email in emails[:]:
        count = 0
        context = {}
        for df in dfs:
            mail = email_col(df)
            df[mail] = df[mail].str.lower().str.replace(' ', '')
            for key in set(df.columns.to_list()).intersection(keys):
                try:
                    context[key] = max(df[df[mail] == email][key], key=len)
                    if key in ['score_test', 'time']:
                        context[key] = df[df[mail] == email].groupby([mail])[key].apply('  \n  '.join)[0]
                    elif key == 'q1videoaudiourls':
                        count += 1
                        context['q' + str(count)] = max(df[df[mail] == email][key], key=len)
                    elif key == 'question':
                        context[key] = df[df[mail] == email].groupby([mail])[key].apply('    \n    '.join)[0]
                except:
                    pass
        try:
            last, name, patronymic = context['name'].split()
        except:
            pass
        context['name'] = max(df0[df0['ma_email'] == email]['name'], key=len)
        context['name_2'] = max(df0[df0['ma_email'] == email]['name_2'], key=len)
        context['name_3'] = max(df0[df0['ma_email'] == email]['name_3'], key=len)
        name = context['name_2'] + ' ' + context['name'] + ' ' + context['name_3']
        context['name'] = name
        context['email'] = email
        context['city'] = re.findall('ship-(.+)/test', max(df0[df0['ma_email'] == email]['referer'], key=len))[0]
        context['novel_score'], context['novel_translated'], context['novel_last'], context['novel_branch'], novel_numbers = novella_insert(email, data, rules)
        cases_pizza_numbers = score_cases_pizza(context)

        k += 1

        context['score'], context['cases_score'] = primary_point(context)
        context.update({key: 'пропуск' for key in keys if key not in context or context[key] == ''})
        context, echelon = context_check(context)
        if values['key'] == 1:
            context['add_id'] = values['add_id']
            context[values['ban_id']] = ''
            
        if echelon != 3:
            doc_disk = DocxTemplate("Ввод/Практикум_шаблон_диск {}.docx".format(values['key']))
            try:
                novel_pizza = web(doc, novel_numbers+cases_pizza_numbers, k)
                novel_pizza_disc = web(doc_disk, novel_numbers+cases_pizza_numbers, k)
            except:
                novel_pizza = web(doc, cases_pizza_numbers, k)
                novel_pizza_disc = web(doc_disk, cases_pizza_numbers, k)

            if echelon in echelons.keys():
                context['novel_pizza'] = novel_pizza_disc
                doc_disk.render({'item': context})
                doc_disk.save('Вывод/'+name+'.docx')
                link = drive_file_create(name, folder_dict[echelon], 'Вывод/'+name+'.docx')
                context['link'] = link
                context['novel_pizza'] = novel_pizza
                echelons[echelon].append(context)
        
        context['k'] = k
        item = [context[col] if col in context.keys() else '-' for col in col_labels]
        tables[echelon].append(item)
        tables['final'].append(item)
        window['progress_bar'].update(round(100*k/len(emails)))
        #
        print(context['name'], echelon)
        #
        
    values['col_labels'] = dict_col_labels.values()
    values['table_1'], values['table_2'], values['table_3'], values['table_final'] = tables.values()
    values['echelon_1'], values['echelon_2'] = echelons.values()
    values['total_count'] = len(emails)

    doc.render(values)
    doc.save("ОтчетПрактикум_{}.docx".format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")))

    for file_name in os.listdir():
        if file_name.startswith('web'):
            os.remove(file_name)

    #send()
    print("Заполнение анкет: %s секунд" % round(time.time() - middle_time))

### Пользовательский интерфейс (Запуск Анкеты)

In [2]:
pd.options.mode.chained_assignment = None
mpltl.rcParams.update({'figure.max_open_warning': 0})

root_id = drive.ListFile({'q': "title = 'Продукты'"}).GetList()[0]['id']
folder_root_id = drive.ListFile({'q': "title = 'Отчеты' and '%s' in parents" % root_id}).GetList()[0]['id']
dict_key = {re.findall('ship-(.+)/test', x)[0]: x for x in list(set(df0['referer'].values)) if 'ship' in x}

sg.theme('Reddit')
window = make_window()
while True:
    event, values = window.read()
    #window['-KEY-'].update(values=sorted(dict_key.keys()))
    if event in (None, 'Exit', 'Выйти'):
        break
    if event == 'Запустить' and values['-KEY-'] != '':
        main(window, values)
window.close()

NameError: name 'pd' is not defined